In [2]:
pip install pandas

     |████████████████████████████████| 11.3 MB 7.6 MB/s eta 0:00:01
     |████████████████████████████████| 15.7 MB 66.8 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install nltk

     |████████████████████████████████| 1.5 MB 8.2 MB/s eta 0:00:01
     |████████████████████████████████| 303 kB 59.3 MB/s eta 0:00:01
     |████████████████████████████████| 745 kB 66.2 MB/s eta 0:00:01
     |████████████████████████████████| 76 kB 14.4 MB/s eta 0:00:01
     |████████████████████████████████| 97 kB 19.7 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install unidecode

     |████████████████████████████████| 241 kB 8.0 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install ftfy

     |████████████████████████████████| 64 kB 3.6 MB/s eta 0:00:011
  Created wheel for ftfy: filename=ftfy-6.0.3-py3-none-any.whl size=41913 sha256=712d2b1fd7f38731c6c84e1ee7d0312923be3e4db96f87842b5286e18e931361
  Stored in directory: /home/jovyan/.cache/pip/wheels/19/f5/38/273eb3b5e76dfd850619312f693716ac4518b498f5ffb6f56d
Successfully built ftfy
Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install gensim

     |████████████████████████████████| 24.0 MB 7.3 MB/s eta 0:00:01
     |████████████████████████████████| 58 kB 10.6 MB/s eta 0:00:01
     |████████████████████████████████| 28.5 MB 59.9 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [11]:
# Import required libraries
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
import unidecode
import unicodedata
import re
import ftfy 
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec

In [12]:
# Import Data
df = pd.read_csv('doc_summaries.csv', encoding = 'Windows-1252')
 
# Check for null values
df[df.isnull().any(axis=1)]
 
# Drop rows with null Values
df.drop(df[df.isnull().any(axis=1)].index,inplace=True)
#df.drop(columns = 'id', inplace = True)
df.head()

,id,text
0,333335d380baf2f45052cd08b2bf268c,american axle manufacturing holdings inc anno...
1,870d67449000455506fc4ca42cef6137,on corporation completed the previously annou...
2,3a0b060acafee4c08eca41044d170642,this presentation contains certain forward lo...
3,8c0d024774a8667ba6a27f9b4b7737ae,net charge off rate is calculated by dividing...
4,809cd0f9013c795c533e67cf99ea8111,englewood inc nasdaq cnob the company or and ...


In [13]:
df['tag'] = df.index
df = df.reset_index()
df = df.drop(['index'], axis = 1)

In [14]:
df['text'] = df['text'].apply(lambda x: ' '.join([word for word in x.split()])) 
df['text'] = df['text'].replace(u'\u201c', '"').replace(u'\u201d', '"')

In [15]:
prev = df['text'].values.tolist()

processed_features = []
for sentence in range(0, len(prev)):
    # Remove all the special characters
    clean = ftfy.fix_text(str(prev[sentence]), uncurl_quotes = True)
    processed_feature = re.sub('[\W_]+', ' ', clean)
     # remove all single characters
    processed_feature= re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_feature)
     # Substituting multiple spaces with single space
    processed_feature = re.sub(r'\s+', ' ', processed_feature, flags=re.I)
    #Removing special characters and sequences
    processed_feature = re.sub(r'\“', '', processed_feature)
    processed_feature = re.sub(r'\”', '', processed_feature)
    processed_feature = re.sub(r'\_', '', processed_feature)
    processed_feature = re.sub(r",", " ", processed_feature )
    processed_feature = re.sub(r"\.", " ", processed_feature )  
    processed_feature = re.sub(r"!", " ", processed_feature )  
    processed_feature = re.sub(r"\(", " ( ", processed_feature )
    processed_feature = re.sub(r"\)", " ) ", processed_feature )
    processed_feature = re.sub(r"\?", " ", processed_feature )
    processed_feature = re.sub(r"\s{2,}", " ", processed_feature )
    processed_feature = re.sub(r"[^A-Za-z0-9(),!.?\'`\"\“\”]", " ", processed_feature )
     # Converting to Lowercase
    processed_feature = processed_feature.lower()
    processed_features.append(processed_feature)

In [16]:
#Creating document list of list   
train_docs = list(map(lambda el:[el], processed_features))

#Tagging document sentences 
comp_docs = [TaggedDocument(
                words=[word for word in document[0].lower().split()],
                tags = [i]
            ) for i, document in enumerate(train_docs)]

In [21]:
max_epoch = 2
vec_size = 20
    
# Train model
model = Doc2Vec(vector_size = vec_size, dm = 0, dbow_words = 1, window = 2, alpha = 0.2)
model.build_vocab(comp_docs)
for epoch in range(max_epoch):
    model.train(comp_docs, total_examples = model.corpus_count, epochs = 3)
    
model.save("Doc2Vec.model")
print("Model Saved")

Model Saved


In [28]:
def build_model(test_doc, compiled_doc):
    '''
    Parameters
    -----------
    test_doc: list of lists - one of the sentence lists
    compiled_doc: list of lists - combined sentence lists to match the index to the sentence 
    '''
    model= Doc2Vec.load("Doc2Vec.model")
    scores = []
    #for doc in test_docs:
    dd = {}
    # Calculate the cosine similarity and return top 25 matches
    score = model.docvecs.most_similar([model.infer_vector(test_docs)],topn=25)
    key = " ".join(test_docs)
    for i in range(len(score)):
    # Get index and score
        x, y = score[i]
        z = df['id'].loc[df.index == x].values[0]
    # Match sentence from other list
        nkey = ' '.join(comp_docs[x][0])
        dd[z, nkey] = y
    scores.append({key:dd})

    return scores

In [25]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [26]:
#Enter test document here 
test_docs = word_tokenize('authorized to take such actions as instructed by any effective date indenture dated as of among hovnanian enterprises by their acceptance of the notes shall be deemed to have instructed and authorized to take such action as instructed by any effective date grant pari passu on such property to for the benefit of the')

In [29]:
#Run model 
build_model(test_docs, comp_docs)

/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  del sys.path[0]


[{'authorized to take such actions as instructed by any effective date indenture dated as of among hovnanian enterprises by their acceptance of the notes shall be deemed to have instructed and authorized to take such action as instructed by any effective date grant pari passu on such property to for the benefit of the': {('edfa358889da47979f31f5da23d3c9db',
    'dish network corporation nasdaq dish dish announced that it has completed its previously announced rights offering to raise of the newly issued shares of class common stock of the shares were issued through the exercise of the subscription rights including the exercise in full by chairman and largest shareholder and of the shares were issued through mr standby purchase commitment this press release shall not constitute an offer to sell or solicitation of an offer to buy any securities nor will there be any sale of the subscription rights class common stock or any other securities in any state or other jurisdiction in which such